# TFLite 변환
주의점: 모든 TF ops가 변환 가능한 것은 아니다. 그럴 때 변환된 모델의 크기를 좀 더 늘려서라도 지원되는
TF ops를 늘리고 싶거나, 직접 정의한 ops를 추가하고 싶다면:
```Python
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

converter.target_spec.experimental_select_user_tf_ops = [
    'your_op_name1',
    'your_op_name2'
]
```

In [1]:
import tensorflow as tf

2023-01-15 03:17:49.657647: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 변환기 빌드 및 옵션 설정
saved_model_dir = "models/2"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# 사전 제공 옵션: DEFAULT(지연과 크기 사이 균형), OPTIMIZE_FOR_LATENCY, OPTIMIZE_FOR_SIZE
converter.optimizations = [tf.lite.Optimize.DEFAULT]

2023-01-15 03:19:21.650291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# 제공한 옵션으로 모델 변환
tflite_model = converter.convert()

In [ ]:
# 파일로 모델 기록
lite_model_dir = "models/lite_model"
with open(lite_model_dir, "wb") as f:
    f.write(tflite_model)

# TFLite 모델 serving
`docker run --use_tflite_model=true` 옵션만 추가하면 가능.

# Prometheus로 모델 서버 모니터링
Serving과 Prometheus 컨테이너를 연다. 프로메테우스가 외부에서 지표 수집 요청을 받으면 프로메테우스는
모델 서버가 공개한 엔드포인트에서 데이터를 받아온다.

1. 먼저 /tmp/prometheus.yml에 구성 정보를 설정한다.
```
global:
  scrape_interval: 15s
  evaluation_interval: 15s
  external_labels:
    monitor: 'tf-serving-monitor'

scrape_configs:
  - job_name: 'prometheus'
    scrape_interval: 5s
    # 아래 경로는 serving에겐 지표를 저장, prometheus에겐 지표 수집 경로이다.
    metrics_path: /monitoring/prometheus/metrics
    static_configs:
    # REST 요청만 가능. 도커 도메인 이름 확인 기능 덕분에 아래 호스트명을 쓰면 8501로 공개된 주소를
    # 자동으로 찾을 수 있다.
    - targets: ['host.docker.internal:8501']
```

2. 프로메테우스 컨테이너 가동
```bash
docker run -p 9090:90090 \
           -v /tmp/prometheus.yml:/etc/prometheus/prometheus.yml \
           -t prom/prometheus
```

3. 서빙 설정
```
prometheus_config {
  enable: true,
  # 아래 경로는, prometheus.yml 속 metrics_path와 동일해야 한다.
  path: "/monitoring/prometheus/metrics"
}
```

4. 서빙 컨테이너 기동
```bash
docker run -p 8501:8501 \
           # source: path to the file or directory on the Docker daemon host.
           # target: the path where the file or directory is mounted in the container.
           --mount type=bind,source=<model_path>,target=<model_path> \
           --mount type=bind,source=<config_path>,target=<config_path> \
           --monitoring_config_file=<config_in_docker_filessystem> \
           -t tensorflow/serving
```